In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [3]:
# list user by id and their rating to each movie 
data=pd.read_csv('movielens1M/ratings.csv')
data.head()


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# list movie by movie_id 
movies=pd.read_csv('movielens1M/movies.csv', encoding='latin-1')
movie_names=pd.DataFrame(movies)
movie_names.head()


,movie_id,movie,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# menggabungkan 2 data frame menjadi 1 / combine two data frames into one  
combined_movies_data=pd.merge(data,movies, on='movie_id')
combined_movies_data.head()

,user_id,movie_id,rating,timestamp,movie,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [6]:
# find movie with must number of views 
rating=pd.DataFrame(combined_movies_data.groupby('movie_id')['rating'].count().sort_values(ascending=False))
rating.head()




,rating
movie_id,
2858,3428
260,2991
1196,2990
1210,2883
480,2672


In [7]:
# view movie title base on highest rating
Filter= combined_movies_data['movie_id']==2858
print(combined_movies_data[Filter]['movie'].unique())

['American Beauty (1999)']


In [8]:
# membuat matriks utility yang akan menjadi matriks crosstab yang dihasilkan dari data film gabungan

In [9]:
# building a utility matrix 
rating_crosstab_mat=combined_movies_data.pivot_table(values='rating',index='user_id', 
columns='movie',fill_value=0)
rating_crosstab_mat.head()


movie,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjü¾¦¼rlighetens kjü¾¼tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Selanjutnya, mengambil matriks utilitas ditransposisikan, yang nantinya menggunakan SVD untuk menguraikannya menjadi representasi sintetis dari ulasan pengguna. Dan menyebutkannya.

In [11]:
# transposing the matrix 
print(rating_crosstab_mat.shape)


(6040, 3706)


In [12]:
X=rating_crosstab_mat.values.T
print(X.shape)


(3706, 6040)


In [13]:
#  menguraikan,buktikan objek SVD, dan kemudian memanggil SVD , kemudian  mengatur matriks resultan untuk memiliki 10 dimensi.

In [14]:
# decomposing the matrix 
SVD=TruncatedSVD(n_components=10,random_state=18)
result_mat=SVD.fit_transform(X)
print(result_mat.shape)


(3706, 10)


In [15]:
corr_mat=np.corrcoef(result_mat)
print(corr_mat.shape)


(3706, 3706)


In [16]:
# Isolating high view movie From the Correlation Matrix 
movies_names=rating_crosstab_mat.columns
movies_list=list(movies_names)
American_Beauty= movies_list.index('American Beauty (1999)')
print(American_Beauty)
corr_American_Beauty=corr_mat[American_Beauty]
print(corr_American_Beauty.shape)

127
(3706,)


In [17]:
#  daftar nama film yang berkorelasi dengan American Beauty (movie dengan rating tertinggi ).

In [18]:
print(list(movies_names[(corr_American_Beauty < 1.0) & (corr_American_Beauty > 0.90)]))


['Almost Famous (2000)', 'American Beauty (1999)', 'American Psycho (2000)', 'Being John Malkovich (1999)', 'Best in Show (2000)', 'Big Kahuna, The (2000)', 'Cell, The (2000)', 'Chasing Amy (1997)', 'Chicken Run (2000)', 'Clerks (1994)', 'Dogma (1999)', 'Election (1999)', 'Eyes Wide Shut (1999)', 'Fight Club (1999)', 'Gladiator (2000)', 'Go (1999)', 'Gun Shy (2000)', 'High Fidelity (2000)', 'Keeping the Faith (2000)', 'Lock, Stock & Two Smoking Barrels (1998)', 'Magnolia (1999)', 'Man on the Moon (1999)', 'Me Myself I (2000)', 'Meet the Parents (2000)', 'Nurse Betty (2000)', 'Original Kings of Comedy, The (2000)', 'Requiem for a Dream (2000)', 'Run Lola Run (Lola rennt) (1998)', 'Rushmore (1998)', 'Shakespeare in Love (1998)', 'Sixth Sense, The (1999)', 'Small Time Crooks (2000)', 'Talented Mr. Ripley, The (1999)', 'Tao of Steve, The (2000)', 'Virgin Suicides, The (1999)', 'What Planet Are You From? (2000)', 'Wonder Boys (2000)']
